# Data Drift and Model Performance Dashboards for Iris dataset

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, model_selection, neighbors

from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab, CatTargetDriftTab
from evidently.tabs import ClassificationPerformanceTab, ProbClassificationPerformanceTab

from evidently.model_profile import Profile
from evidently.profile_sections import DataDriftProfileSection, CatTargetDriftProfileSection 
from evidently.profile_sections import ClassificationPerformanceProfileSection, ProbClassificationPerformanceProfileSection

## Iris data

In [ ]:
iris = datasets.load_iris()

In [ ]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

## Data Drift Dashboard

In [ ]:
iris_data_drift_report = Dashboard(tabs=[DataDriftTab])
iris_data_drift_report.calculate(iris_frame, iris_frame, column_mapping = None)

In [ ]:
iris_data_drift_report.show()

In [ ]:
#iris_data_drift_report.save('iris_data_drift.html')

## Data Drift Profile

In [ ]:
iris_data_drift_profile = Profile(sections=[DataDriftProfileSection])
iris_data_drift_profile.calculate(iris_frame, iris_frame, column_mapping = None)

In [ ]:
drift_profile = iris_data_drift_profile.json() 

In [ ]:
drift_profile

## Data and Target Drift Dashboard

In [ ]:
iris_frame['target'] = iris.target

In [ ]:
iris_data_and_target_drift_report = Dashboard(tabs=[DataDriftTab, CatTargetDriftTab])
iris_data_and_target_drift_report.calculate(iris_frame[:75], iris_frame[75:], column_mapping = None)

In [ ]:
iris_data_and_target_drift_report.show()

In [ ]:
iris_data_and_target_drift_report.save('iris_data_and_target_drift.html')

## Data and Target Drift Profile

In [ ]:
iris_target_and_data_drift_profile = Profile(sections=[DataDriftProfileSection, CatTargetDriftProfileSection])
iris_target_and_data_drift_profile.calculate(iris_frame[:75], iris_frame[75:], column_mapping = None) 

In [ ]:
data_and_target_profile = iris_target_and_data_drift_profile.json() 

In [ ]:
data_and_target_profile

## Model Performance Dashboard

In [ ]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

In [ ]:
reference, production, y_train, y_test = model_selection.train_test_split(iris_frame, 
                                                                          iris.target, 
                                                                          random_state=0)

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors=1)

In [ ]:
model.fit(reference, y_train)

In [ ]:
train_predictions = model.predict(reference)
test_predictions = model.predict(production)

In [ ]:
reference['target'] = y_train
reference['prediction'] = train_predictions

production['target'] = y_test
production['prediction'] = test_predictions

In [ ]:
reference.target = reference.target.apply(lambda x: iris.target_names[x])
reference.prediction = reference.prediction.apply(lambda x: iris.target_names[x])

production.target = production.target.apply(lambda x: iris.target_names[x])
production.prediction = production.prediction.apply(lambda x: iris.target_names[x])

In [ ]:
production.head(3)

In [ ]:
iris_column_mapping = {}

iris_column_mapping['target'] = 'target'
iris_column_mapping['prediction'] = 'prediction'
iris_column_mapping['numerical_features'] = iris.feature_names

In [ ]:
iris_model_performance = Dashboard(tabs=[ClassificationPerformanceTab])
iris_model_performance.calculate(reference, production, column_mapping = iris_column_mapping)
iris_model_performance.show()

In [ ]:
iris_model_performance.save('iris_classification_performance.html')

## Model Performance Profile

In [ ]:
iris_classification_performance_profile = Profile(sections=[ClassificationPerformanceProfileSection])
iris_classification_performance_profile.calculate(reference, production, column_mapping = iris_column_mapping)

In [ ]:
classification_performance_profile = iris_classification_performance_profile.json() 

In [ ]:
classification_performance_profile

## Probabilistic Model Performance Dashboard

In [ ]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

In [ ]:
reference, production, y_train, y_test = model_selection.train_test_split(iris_frame, 
                                                                          iris.target, 
                                                                          random_state=0)

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors=5)

In [ ]:
model.fit(reference, y_train)

In [ ]:
train_probas = pd.DataFrame(model.predict_proba(reference))
train_probas.columns = iris.target_names
test_probas = pd.DataFrame(model.predict_proba(production))
test_probas.columns = iris.target_names

In [ ]:
reference['target'] = [iris.target_names[x] for x in y_train]
production['target'] = [iris.target_names[x] for x in y_test]

In [ ]:
reference.reset_index(inplace=True, drop=True)
reference['result'] = [iris.target_names[x] for x in y_train]
merged_reference = pd.concat([reference, train_probas], axis = 1)

production.reset_index(inplace=True, drop=True)
production['result'] = [iris.target_names[x] for x in y_test]
merged_production = pd.concat([production, test_probas], axis = 1)

In [ ]:
iris_column_mapping = {}

iris_column_mapping['target'] = 'target'
iris_column_mapping['prediction'] = iris.target_names.tolist()
iris_column_mapping['numerical_features'] = iris.feature_names

In [ ]:
iris_model_performance = Dashboard(tabs=[ProbClassificationPerformanceTab])
iris_model_performance.calculate(merged_reference, merged_production, column_mapping = iris_column_mapping)
iris_model_performance.show()

## Probabilistic Model Performance Profile

In [ ]:
iris_prob_performance_profile = Profile(sections=[ProbClassificationPerformanceProfileSection])
iris_prob_performance_profile.calculate(merged_reference, merged_production, column_mapping = iris_column_mapping)

In [ ]:
prob_performance_profile = iris_prob_performance_profile.json() 

In [ ]:
prob_performance_profile